# PLN - Pré-processamento de texto
Os passos da *pipeline* foram baseados no seguinte artigo:
- [https://sol.sbc.org.br/index.php/stil/article/download/31163/30966/](https://sol.sbc.org.br/index.php/stil/article/download/31163/30966/)

In [16]:
import pandas as pd
import nltk
from nltk.corpus import stopwords

# Importar o corpus de um arquivo CSV
df = pd.read_csv('data/DepreRedditBR.csv')

# Flag utilizada para definir se a ferramenta Enelvo será utilizada ou não
USE_ENELVO = False

df.shape

(509669, 1)

## Exibir o dataframe

In [17]:
df.head() # Exibir as primeiras linhas do dataframe

,text
0,"Parece que ninguem quer ficar perto de mim, e,..."
1,"Eu sou deprimido, porem tem períodos que fico ..."
2,"Boa tarde. Bem, é basicamente um desabafo. A v..."
3,"Me tirem uma duvida, ser perfeccionista ao ac..."
4,"Oi gente, desculpa o textão, já mandei isso em..."


In [18]:
# Obter uma amostra do corpus (frac -> de 0 a 1)
df = df.sample(frac=0.015, random_state=42)

# Obter a única coluna do dataframe: text
texts = df.text

In [19]:
texts

208601    não há nada que eu queira mais do que voltar n...
462528    Lidando com mudanças de humor Tenho tido muita...
76680     Se sua noiva não usar com voce, ja sabe. Não p...
359032    muitas vezes não consigo trabalhar cerca de um...
353889    quais são suas ações quando você está se senti...
                                ...                        
285128    Estou deprimido há mais tempo do que me lembro...
244216    efeitos colaterais do wellbutrin, espero não e...
130960                                       Q fita em mano
307709    lidar com a ansiedade enquanto o marido está f...
48159     Seu namorado está com dependência emocional de...
Name: text, Length: 7645, dtype: object

## Opcional - Utilizano o Enelvo para normalizar aspectos do corpus
A normalização com Enelvo trata de aspectos como erros ortográficos, gírias, siglas e outros detalhes.

A substituição das virgulas por uma *label* temporária é feita para contornar 

In [20]:

if USE_ENELVO:
    # Enelvo (https://thalesbertaglia.com/enelvo/)
    from enelvo.normaliser import Normaliser

    # Substituir as vírgulas pela label 'chavevirg' para evitar conflitos durante a normalização do texto
    texts = texts.str.replace(r',', 'chavevirg')
    texts.values

    # Criar uma instância do normalizador
    normaliser_instance = Normaliser(tokenizer='readable')
    # Aplicar a normalização com Enelvo para os textos do corpus
    texts = texts.apply(lambda text: normaliser_instance.normalise(text))

## Remoção de URLs

In [21]:
import re
# Remove URLs if there is any
texts = texts.apply(lambda text: re.sub(r'http\S+', '', text))

if USE_ENELVO:
    # Reverter a substituição de vírgulas
    texts = texts.str.replace('chavevirg', ',')

texts

208601    não há nada que eu queira mais do que voltar n...
462528    Lidando com mudanças de humor Tenho tido muita...
76680     Se sua noiva não usar com voce, ja sabe. Não p...
359032    muitas vezes não consigo trabalhar cerca de um...
353889    quais são suas ações quando você está se senti...
                                ...                        
285128    Estou deprimido há mais tempo do que me lembro...
244216    efeitos colaterais do wellbutrin, espero não e...
130960                                       Q fita em mano
307709    lidar com a ansiedade enquanto o marido está f...
48159     Seu namorado está com dependência emocional de...
Name: text, Length: 7645, dtype: object

## Remoção de pontuação
A remoção de pontuação visa a redução do ruído (mantendo apenas as palavras que importam) e aumenta a eficiência do processamento.
É importante frisar que a pontuação **NÃO** é removida quando faz parte de *hashtags*, datas ou números com vírgula.

In [22]:
def remove_punctuation(text: str):
    return re.sub(
        r'(?<!\d),(?=\D)|(?<=\D),(?!\d)|(?<!\d),(?=\d)|(?<!\d)\/|\/(?!\d)|_|[^\w#\/\s,\@]','', text)

texts = texts.apply(lambda text: remove_punctuation(text))
texts.values

array(['não há nada que eu queira mais do que voltar no tempo e refazer minha vida com todas as memórias e conhecimentos que tenho agora eu sonho acordado com isso constantemente é a única coisa que me dá uma aparência de felicidade Na verdade eu era feliz quando criança tenho boas lembranças de brincar com meus amigos desenhar assistir desenhos animados etc e parte meu coração saber que nunca mais serei feliz de verdade assim se eu tivesse que adivinhar onde tudo deu errado provavelmente foi quando fui separado de meus amigos no ensino fundamental Desenvolvi ansiedade geralsocial e depressão se eu pudesse retroceder minha vida não deixaria que isso me afetasse tanto faria novos bons amigos eu também levaria o desenho mais a sério queria trabalhar com animação mas minha anedonia tirou toda a minha motivação Além disso fico muito desanimado quando vejo alguém que é muito melhor do que eu Eu também leria mais livros assistiria mais filmesprogramas etc eu sei que poderia fazer isso agora 

## Tokenização do texto
Após a remoção de partes menos relevantes dos textos pertencentes ao *corpus*, é importante tokenizar tais textos, permitindo a aplicação de técnicas como a contagem de vocabulário. Além disso, isso facilita o processamento computacional.

In [23]:
from nltk.tokenize import word_tokenize

nltk.download('punkt_tab')
texts = texts.apply(lambda text: word_tokenize(text))
texts

[nltk_data] Downloading package punkt_tab to /home/yago/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


208601    [não, há, nada, que, eu, queira, mais, do, que...
462528    [Lidando, com, mudanças, de, humor, Tenho, tid...
76680     [Se, sua, noiva, não, usar, com, voce, ja, sab...
359032    [muitas, vezes, não, consigo, trabalhar, cerca...
353889    [quais, são, suas, ações, quando, você, está, ...
                                ...                        
285128    [Estou, deprimido, há, mais, tempo, do, que, m...
244216    [efeitos, colaterais, do, wellbutrin, espero, ...
130960                                  [Q, fita, em, mano]
307709    [lidar, com, a, ansiedade, enquanto, o, marido...
48159     [Seu, namorado, está, com, dependência, emocio...
Name: text, Length: 7645, dtype: object

## Remoção de *stopwords*
É necessário remover palavras muito comuns no idioma, visto que, geralmente, elas trazem pouco significado semântico. Isso ajuda a **reduzir a dimensionalidade** e a aumentar a ênfase do vocabulário em **palavras mais importantes**.

Exemplos: artigos, preposições, conjunções, pronomes, advérbios de frequência e modo etc.

In [ ]:
nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('portuguese')